In [1]:
import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(784, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [2]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

In [3]:
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [4]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [5]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

In [6]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [7]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/50
235/235 [==============================] - 2s 5ms/step - loss: 0.3826 - val_loss: 0.1940
Epoch 2/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1835 - val_loss: 0.1526
Epoch 3/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1478 - val_loss: 0.1326
Epoch 4/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1303 - val_loss: 0.1201
Epoch 5/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1193 - val_loss: 0.1117
Epoch 6/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1113 - val_loss: 0.1059
Epoch 7/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1059 - val_loss: 0.1017
Epoch 8/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1020 - val_loss: 0.0988
Epoch 9/50
235/235 [==============================] - 1s 2ms/step - loss: 0.0996 - val_loss: 0.0967
Epoch 10/50
235/235 [==============================] - 1s 2ms/step - loss: 0.0975 - val_loss: 0.0955

In [9]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [10]:
len(encoded_imgs[0])

32

In [12]:
len(decoded_imgs[0])

784